<a href="https://colab.research.google.com/github/surjit84/ApacheBeamHandsOn/blob/main/apache_beam_demo02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 57.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 31.6 M

In [6]:
import apache_beam as beam
p1 = beam.Pipeline()
lines =(
    p1
    # Creating List
    |beam.Create(['a','b','c','d','e'])
    |beam.io.WriteToText('data/output01')
)
p1.run()
# Visualize the output
!{('head -n 20 data/output01-00000-of-00001')}

a
b
c
d
e


In [7]:
import apache_beam as beam
p1 = beam.Pipeline()
lines =(
    p1
    #Creating Tuple
    |beam.Create([('a',1),('b',2),('c',3),('d',4),('e',5)])
    |beam.io.WriteToText('data/output02')
)
p1.run()
# Visualize the output
!{('head -n 20 data/output02-00000-of-00001')}

('a', 1)
('b', 2)
('c', 3)
('d', 4)
('e', 5)


In [32]:
import apache_beam as beam
#custom function to be used on dictionary
def dic_cal(tempdt):
  retdt = {}
  for k, v in tempdt.items():
    retdt[k]= [i*3 for i in v]
  return retdt

def tup_cal(temptup):
  return (temptup[0],[i*3 for i in temptup[1]])

p1 = beam.Pipeline()
lines =(
    p1
    #Creating Dictionary
    |beam.Create({'a':[1,2,3,4,5],
     'b':[5,6,7,8],
      'c':[3,7,8,9,10],
       'd':[4,7,2,3,1],
        'e':[5,9,5,6,2,15]})
    |beam.Map(lambda record: tup_cal(record))
    #|beam.Map(lambda element: element)
    |beam.io.WriteToText('data/output03')
)
p1.run()
# Visualize the output
!{('head -n 20 data/output03-00000-of-00001')}

('a', [3, 6, 9, 12, 15])
('b', [15, 18, 21, 24])
('c', [9, 21, 24, 27, 30])
('d', [12, 21, 6, 9, 3])
('e', [15, 27, 15, 18, 6, 45])


In [31]:
def dic_cal(tempdt):
  retdt = {}
  for k, v in tempdt.items():
    retdt[k]= [i*3 for i in v]
  return retdt
def tup_cal(temptup):
  return (temptup[0],[i*3 for i in temptup[1]])
record = {'a':[1,2,3,4,5],
     'b':[5,6,7,8],
      'c':[3,7,8,9,10],
       'd':[4,7,2,3,1],
        'e':[5,9,5,6,2,15]}
record1=('a', [1, 2, 3, 4, 5])
#dic_cal(record)
tup_cal(record1)

('a', [3, 6, 9, 12, 15])

In [19]:
record = {'a':[1,2,3,4,5],
     'b':[5,6,7,8],
      'c':[3,7,8,9,10],
       'd':[4,7,2,3,1],
        'e':[5,9,5,6,2,15]}
type(record)

dict

Branching

In [35]:
import apache_beam as beam
p1 = beam.Pipeline()
input_collection =(
    p1
    |"Read from Text file" >> beam.io.ReadFromText('dept_data.txt')
    |"Split record with comma seperator" >> beam.Map(lambda record: record.split(','))
)
attendance_count =(
    input_collection
    |"Filter record for Account" >> beam.Filter(lambda record: record[3]=='Accounts')
    |"Map account key with value 1" >> beam.Map(lambda record: (record[1], 1))
    |"Count sum of account key" >> beam.CombinePerKey(sum)
    |"Write output to account_output" >> beam.io.WriteToText('data/account_output')
)
hr_count =(
    input_collection
    |"Filter record for HR" >> beam.Filter(lambda record: record[3]=='HR')
    |"Map hr key with value 1" >> beam.Map(lambda record: (record[1], 1))
    |"Count sum of hr key" >> beam.CombinePerKey(sum)
    |"Write output to hr_output" >> beam.io.WriteToText('data/hr_output')
)
p1.run()

In [37]:
print('Account count:')
!{('head -n 20 data/account_output-00000-of-00001')}
print('HR count:')
!{('head -n 20 data/hr_output-00000-of-00001')}

Account count:
('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)
HR count:
('Beryl', 62)
('Olga', 31)
('Leslie', 31)
('Mindy', 31)
('Vicky', 31)
('Richard', 31)
('Kirk', 31)
('Kaori', 31)
('Oscar', 31)


Flatten two PCollection

In [38]:
import apache_beam as beam
p1 = beam.Pipeline()
input_collection =(
    p1
    |"Read from Text file" >> beam.io.ReadFromText('dept_data.txt')
    |"Split record with comma seperator" >> beam.Map(lambda record: record.split(','))
)
attendance_count =(
    input_collection
    |"Filter record for Account" >> beam.Filter(lambda record: record[3]=='Accounts')
    |"Map account key with value 1" >> beam.Map(lambda record: ('Account, ' + record[1], 1))
    |"Count sum of account key" >> beam.CombinePerKey(sum)
    #|"Write output to account_output" >> beam.io.WriteToText('data/account_output')
)
hr_count =(
    input_collection
    |"Filter record for HR" >> beam.Filter(lambda record: record[3]=='HR')
    |"Map hr key with value 1" >> beam.Map(lambda record: ('HR, ' + record[1], 1))
    |"Count sum of hr key" >> beam.CombinePerKey(sum)
    #|"Write output to hr_output" >> beam.io.WriteToText('data/hr_output')
)
final_output =(
    (attendance_count,hr_count)
    |"Combine both" >> beam.Flatten()
    |"Write combined output together" >> beam.io.WriteToText('data/both_output')
)
p1.run()

In [39]:
!{('head -n 20 data/both_output-00000-of-00001')}

('Account, Marco', 31)
('Account, Rebekah', 31)
('Account, Itoe', 31)
('Account, Edouard', 31)
('Account, Kyle', 62)
('Account, Kumiko', 31)
('Account, Gaston', 31)
('Account, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


ParDo and DoFn Class

In [43]:
import apache_beam as beam
class SplitFn(beam.DoFn):
  def process(self, element):
    return [element.split(',')]

class FilterAccount(beam.DoFn):
  def process(self, element):
    if element[3]=='Account':
      return [element]

class PairEmployee(beam.DoFn):
  def process(self, element):
    return [(element[3]+" ,"+element[1], 1)]
p1 = beam.Pipeline()
attendance_count =(
    p1
    |beam.io.ReadFromText('dept_data.txt')
    # ParDo as Custom Processor
    |beam.ParDo(SplitFn())
    # ParDo as Filter
    |beam.ParDo(FilterAccount())
    # ParDo as Map
    |beam.ParDo(PairEmployee())
    |beam.CombinePerKey(sum)
    |beam.io.WriteToText('data/pardo_output')
)
p1.run()

In [ ]:
!{('head -n 20 data/pardo_output-00000-of-00001')}